In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Function to scrape data from a given page URL
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table with class "table"
    table = soup.find('table', class_='table')

    # Extracting data from each row
    rows = table.find_all('tr')

    # Initialize empty lists for each column
    columns = []
    data = []

    # Extracting column names
    header_row = rows[0]
    header_cells = header_row.find_all('th')
    for header_cell in header_cells:
        columns.append(header_cell.get_text(strip=True))

    # Extracting data from each row
    for row in rows[1:]:
        row_data = []
        cells = row.find_all(['th', 'td'])
        for cell in cells:
            row_data.append(cell.get_text(strip=True))
        data.append(row_data)

    # Create a DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

# URL for the first page
base_url = 'https://www.scrapethissite.com/pages/forms'

# Initialize df_all before the loop
df_all = pd.DataFrame()

# Loop over different values of per_page
for per_page in [25, 50, 100]:
    # Set num_pages based on per_page
    if per_page == 25:
        num_pages = 24
    elif per_page == 50:
        num_pages = 12
    elif per_page == 100:
        num_pages = 6
    # Scrape pages based on per_page value
    for page_num in range(1, num_pages + 1):
        url = f'{base_url}/?page_num={page_num}&per_page={per_page}'
        df_page = scrape_page(url)
        df_all = pd.concat([df_all, df_page], ignore_index=True)

# Display the final DataFrame
df_all


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,,0.55,299,264,35
1,Buffalo Sabres,1990,31,30,,0.388,292,278,14
2,Calgary Flames,1990,46,26,,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,,0.425,273,298,-25
...,...,...,...,...,...,...,...,...,...
1741,Tampa Bay Lightning,2011,38,36,8,0.463,235,281,-46
1742,Toronto Maple Leafs,2011,35,37,10,0.427,231,264,-33
1743,Vancouver Canucks,2011,51,22,9,0.622,249,198,51
1744,Washington Capitals,2011,42,32,8,0.512,222,230,-8


In [3]:
df_all.to_csv('data/Hockey.csv', index = False)